In [1]:
#libs for reading data
import xarray as xr
import gcsfs
import glob
import numpy as np
import matplotlib.pyplot as plt
from xhistogram.xarray import histogram

# read in old data & get last time
- just increment by one so that slice won't repeat data

In [2]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'
fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)
ds_old = xr.open_zarr(fs.get_mapper(zstore), consolidated=True,decode_cf=False)
ds_old.close()
lasttime=ds_old.time[-1] + 1
print('last time incremented up 1',lasttime.data)
ds_old

last time incremented up 1 294835.0


,Array,Chunk
Bytes,174.38 GB,226.08 MB
Shape,"(48208, 628, 1440)","(2000, 157, 180)"
Count,801 Tasks,800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,174.38 GB,226.08 MB
Shape,"(48208, 628, 1440)","(2000, 157, 180)"
Count,801 Tasks,800 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [3]:
#read in new data
ds=xr.open_dataset('../../ccmp_data/ccmp.nc',decode_cf=False,mask_and_scale=False)
ds.close()
ds_all = ds
#remove any duplicates
_, index = np.unique(ds_all['time'], return_index=True)
ds_all=ds_all.isel(time=index)
#rechunck data  #data in int16 = 2 bytes 
itime_chunk = 2000   #200
ilat_chunk = 157    #300
ilon_chunk = 180    #600
ds_all = ds_all.chunk({'time':itime_chunk,'latitude':ilat_chunk,'longitude':ilon_chunk})
ds_new = ds_all.sel(time=slice(lasttime,9999999))
ds_new = xr.decode_cf(ds_new)

In [4]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'
fs_out = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)
ds_new.to_zarr(fs_out.get_mapper(zstore), mode='a',append_dim='time', consolidated=True) 

OSError: Forbidden: https://www.googleapis.com/upload/storage/v1/b/pangeo-nasa-ccmp/o
pangeo@pangeo-181919.iam.gserviceaccount.com does not have storage.objects.create access to the Google Cloud Storage object.

# test new data

In [ ]:
zstore = 'gs://pangeo-nasa-ccmp/zarr'
fs = gcsfs.GCSFileSystem(project='pangeo-181919',requester_pays=True)
ds = xr.open_zarr(fs.get_mapper(zstore), consolidated=True)
ds